In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 视频仓库Curl演示

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vision/video_warehouse_curl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vision/video_warehouse_curl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>                                                                                         
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/vision/video_warehouse_curl.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

学习如何发出对[视频仓库](https://cloud.google.com/vision-ai/docs)的curl请求。

### 目标
本文的目标是演示如何向视频仓库发出curl请求。

curl示例按资源类型进行组织: 语料库，资产，数据架构，注释，集合。

### 成本

本教程使用 Google Cloud 的可收费组件：

Vertex AI Vision（[定价](https://cloud.google.com/vision-ai/pricing)）

在开始之前

设置您的Google Cloud项目

无论您使用的是哪种笔记本环境，下面的步骤都是必需的。

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得300美元的免费信用额度用于计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 验证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

1. 点云人工智能工作台
* 不需要操作，因为您已经通过身份验证。

2. 本地JupyterLab实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

In [ ]:
# ! gcloud auth application-default login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user(project_id=PROJECT_ID)

### 设置其他常数

In [ ]:
ENDPOINT_WITH_VERSION = (
    "https://warehouse-visionai.googleapis.com/v1"  # @param {type: "string"}
)
PROJECT_NUMBER_STR = !gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
PROJECT_NUMBER = int(PROJECT_NUMBER_STR[0])
LOCATION_ID = "us-central1"  # @param ["us-central1"]

# 语料库管理

创建语料库

In [ ]:
# @title Create VOD corpus
CORPUS_DISPLAY_NAME="Test corpus" # @param {type: "string"}
CORPUS_DESCRIPTION="Test corpus description" # @param {type: "string"}
REQUEST="""{
  'display_name': '%s',
  'description': '%s',
  'type': 'VIDEO_ON_DEMAND',
  'search_capability_setting' : {
    'search_capabilities': {
      'type':'EMBEDDING_SEARCH'
    }
  }
}"""%(CORPUS_DISPLAY_NAME, CORPUS_DESCRIPTION)
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" -X POST -d "$REQUEST"

In [ ]:
# @title Poll create corpus operation
CREATE_CORPUS_OPERATION_ID="" # @param {type: "string"}
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/warehouseOperations/$CREATE_CORPUS_OPERATION_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

获取语料库

In [ ]:
CORPUS_ID="" # @param {type: "string"}
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

## 列出语料库

In [ ]:
# @title List all corpora under one project.
!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List all corpora with page size specified.
PAGE_SIZE=2 # @param {type: "integer"}
!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora?'page_size='{PAGE_SIZE} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List copora with page token specified.
PAGE_SIZE=2 # @param {type: "integer"}
PAGE_TOKEN="" # @param {type: "string"}
!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora?'page_size='{PAGE_SIZE}'&page_token='{PAGE_TOKEN} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List corpus filter by certain type (type!=, type=).
TYPE='VIDEO_ON_DEMAND' # @param {type: "string"}
PAGE_SIZE=2
!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora?'page_size='{PAGE_SIZE}'&filter=type!='{TYPE} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora?'page_size='{PAGE_SIZE}'&filter=type='{TYPE} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

## 更新语料库

In [ ]:
# @title Use update mask * to do a full replacement
CORPUS_ID="" # @param {type: "string"}
UPDATE_MASK="*"
REQUEST="""{
  'display_name': 'test-update',
  'description': 'update description',
}"""

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

In [ ]:
# @title Error: TTL is not allowed to be updated for VOD
CORPUS_ID="" # @param {type: "string"}
UPDATE_MASK="display_name,default_ttl"
REQUEST="""{
  'display_name':'updated',
  'description': 'sample description_ignore',
  'default_ttl': {'seconds':'1000000'}}"""

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

In [ ]:
# @title Error: search_capability_setting is not allowed to be updated for VOD.
CORPUS_ID="" # @param {type: "string"}
UPDATE_MASK="search_capability_setting"
REQUEST="""{
  'display_name': 'test-update',
  'description': 'update description',
  'search_capability_setting': { 'search_capabilities': { 'type': 'EMBEDDING_SEARCH'} }
}"""

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID?'update_mask='{UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

删除语料库##

In [ ]:
# @title Please run other sections first which requires a valid Corpus before delete the corpus.
CORPUS_ID="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X DELETE

资产管理

创建资产

In [ ]:
# @title CreateAsset (System generates asset Id)
CORPUS_ID="" # @param {type: "string"}
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" -X POST

In [ ]:
# @title CreateAsset (User specified asset Id)
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets?asset_id={ASSET_ID} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" -X POST

上传资产

In [ ]:
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
GCS_URI="gs://cloud-samples-data/video/animals.mp4" # @param {type: "string"}
REQUEST="""{
  'asset_source': {
    'asset_gcs_source': {
      'gcs_uri': '%s'}}
}"""%(GCS_URI)
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID:upload \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X POST \
  -d "$REQUEST"

In [ ]:
# @title Poll upload asset operation
CORPUS_ID="" # @param {type: "string"}
UPLOAD_ASSET_OPERATION_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/operations/$UPLOAD_ASSET_OPERATION_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

生成检索URL

In [ ]:
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID:generateRetrievalUrl \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X POST

## 分析资产

In [ ]:
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID:analyze \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X POST

In [ ]:
# @title Poll analyze asset operation
CORPUS_ID="" # @param {type: "string"}
ANALYZE_ASSET_OPERATION_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/operations/$ANALYZE_ASSET_OPERATION_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

获取资产

In [ ]:
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

## 列出资产

In [ ]:
# @title List all assets under one corpus.
CORPUS_ID="" # @param {type: "string"}

!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List all assets with page size specified.
CORPUS_ID="" # @param {type: "string"}
PAGE_SIZE=2 # @param {type: "integer"}
!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets?'page_size='{PAGE_SIZE} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List assets with page token specified.
CORPUS_ID="" # @param {type: "string"}
PAGE_SIZE=2 # @param {type: "integer"}
PAGE_TOKEN="" # @param {type: "string"}
!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets?'page_size='{PAGE_SIZE}'&page_token='{PAGE_TOKEN} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List assets filter by has content or not (assets_with_contents = true, assets_with_contents = false).
CORPUS_ID="" # @param {type: "string"}
PAGE_SIZE=2 # @param {type: "integer"}
!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets?'page_size='{PAGE_SIZE}'&filter=assets_with_contents=true' \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets?'page_size='{PAGE_SIZE}'&filter=assets_with_contents=false' \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

更新资产

VOD资产中的任何内容都无法更新。预计所有下面的更新都会失败。

In [ ]:
# @title TTL is not allowed to be updated.
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
UPDATE_MASK="*" # @param {type: "string"}
REQUEST="""{
  'ttl': {'seconds':'1000000'},
}"""

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

In [ ]:
# @title asset_gcs_source is output only, not allowed to be updated.
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
UPDATE_MASK="*" # @param {type: "string"}
REQUEST="""{
  'asset_gcs_source': {
    'gcs_uri': 'gs://cloud-samples-data/video/cat2.mp4'
  }
}"""

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

删除资产

In [ ]:
# @title Please run other sections first which requires a valid Asset before delete the asset.

CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X DELETE

In [ ]:
# @title Poll delete asset operation
CORPUS_ID="" # @param {type: "string"}
DELETE_ASSET_OPERATION_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/operations/$DELETE_ASSET_OPERATION_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

数据模式管理

创建数据架构

In [ ]:
# @title CreateDataSchema (primitive data type)
CORPUS_ID="" # @param {type: "string"}
DATA_SCHEMA_KEY="test" # @param {type: "string"}
DATA_SCHEMA_TYPE="STRING" # @param {type: "string"}
DATA_SCHEMA_GRANULARITY="GRANULARITY_PARTITION_LEVEL" # @param {type: "string"}
DATA_SCHEMA_SEARCH_STRATEGY_TYPE="SMART_SEARCH" # @param {type: "string"}

REQUEST="""{
  'key': '%s',
  'schema_details' : {
    'type':'%s',
    'granularity':'%s',
    'search_strategy': {
      'search_strategy_type':'%s'
    }
  }
}"""%(DATA_SCHEMA_KEY, DATA_SCHEMA_TYPE, DATA_SCHEMA_GRANULARITY,DATA_SCHEMA_SEARCH_STRATEGY_TYPE)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/dataSchemas \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" -X POST -d "$REQUEST"

In [ ]:
# @title CreateDataSchema (customized struct data type)
CORPUS_ID="" # @param {type: "string"}
DATA_SCHEMA_KEY="tutorial" # @param {type: "string"}
DATA_SCHEMA_GRANULARITY="GRANULARITY_PARTITION_LEVEL" # @param {type: "string"}
DATA_SCHEMA_FIELD_KEY_1="text" # @param {type: "string"}
DATA_SCHEMA_FIELD_TYPE_1="STRING" # @param {type: "string"}
DATA_SCHEMA_FIELD_SEARCH_STRATEGY_TYPE_1="SMART_SEARCH" # @param {type: "string"}
DATA_SCHEMA_FIELD_KEY_2="score" # @param {type: "string"}
DATA_SCHEMA_FIELD_TYPE_2="FLOAT" # @param {type: "string"}
DATA_SCHEMA_FIELD_SEARCH_STRATEGY_TYPE_2="EXACT_SEARCH" # @param {type: "string"}

REQUEST="""{
  'key': '%s',
  'schema_details' : {
    'type':'CUSTOMIZED_STRUCT',
    'granularity':'%s',
    'customized_struct_config': {
      'field_schemas': {
         '%s': {
            'type':'%s',
            'granularity':'%s',
            'search_strategy': {
               'search_strategy_type':'%s'
            }
         },
         '%s': {
            'type':'%s',
            'granularity':'%s',
            'search_strategy': {
               'search_strategy_type':'%s'
            }
         }
      }
    }
  }
}"""%(DATA_SCHEMA_KEY, DATA_SCHEMA_GRANULARITY, DATA_SCHEMA_FIELD_KEY_1, DATA_SCHEMA_FIELD_TYPE_1,DATA_SCHEMA_GRANULARITY,DATA_SCHEMA_FIELD_SEARCH_STRATEGY_TYPE_1, DATA_SCHEMA_FIELD_KEY_2, DATA_SCHEMA_FIELD_TYPE_2,DATA_SCHEMA_GRANULARITY, DATA_SCHEMA_FIELD_SEARCH_STRATEGY_TYPE_2)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/dataSchemas \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" -X POST -d "$REQUEST"

获取数据模式

In [ ]:
CORPUS_ID="" # @param {type: "string"}
DATA_SCHEMA_KEY="test" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/dataSchemas/$DATA_SCHEMA_KEY \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

## 列出数据模式

In [ ]:
# @title List all data schemas under one corpus.
CORPUS_ID="" # @param {type: "string"}

!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/dataSchemas \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List all data schemas with page size specified.
CORPUS_ID="" # @param {type: "string"}
PAGE_SIZE=2 # @param {type: "integer"}

!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/dataSchemas?'page_size='{PAGE_SIZE} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List data schemas with page token specified.
CORPUS_ID="" # @param {type: "string"}
PAGE_SIZE=2 # @param {type: "integer"}
PAGE_TOKEN="" # @param {type: "string"}

!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/dataSchemas?'page_size='{PAGE_SIZE}'&page_token='{PAGE_TOKEN} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

更新数据模式

In [ ]:
# @title Use update mask * to do a full replacement
CORPUS_ID="" # @param {type: "string"}
DATA_SCHEMA_KEY="test" # @param {type: "string"}
DATA_SCHEMA_TYPE="STRING" # @param {type: "string"}
DATA_SCHEMA_GRANULARITY="GRANULARITY_PARTITION_LEVEL" # @param {type: "string"}
DATA_SCHEMA_SEARCH_STRATEGY_TYPE="SMART_SEARCH" # @param {type: "string"}
UPDATE_MASK="*"

REQUEST="""{
  'key': '%s',
  'schema_details' : {
    'type':'%s',
    'granularity':'%s',
    'search_strategy': {
      'search_strategy_type':'%s'
    }
  }
}"""%(DATA_SCHEMA_KEY, DATA_SCHEMA_TYPE, DATA_SCHEMA_GRANULARITY,DATA_SCHEMA_SEARCH_STRATEGY_TYPE)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/dataSchemas/$DATA_SCHEMA_KEY?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

In [ ]:
# @title Use update mask to do partial replacement
CORPUS_ID="" # @param {type: "string"}
DATA_SCHEMA_KEY="test" # @param {type: "string"}
DATA_SCHEMA_TYPE="FLOAT" # @param {type: "string"}
DATA_SCHEMA_GRANULARITY="GRANULARITY_ASSET_LEVEL" # @param {type: "string"}
DATA_SCHEMA_SEARCH_STRATEGY_TYPE="SMART_SEARCH" # @param {type: "string"}
UPDATE_MASK="schema_details.granularity" # @param {type: "string"}

REQUEST="""{
  'key': '%s',
  'schema_details' : {
    'type':'%s',
    'granularity':'%s',
    'search_strategy': {
      'search_strategy_type':'%s'
    }
  }
}"""%(DATA_SCHEMA_KEY, DATA_SCHEMA_TYPE, DATA_SCHEMA_GRANULARITY,DATA_SCHEMA_SEARCH_STRATEGY_TYPE)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/dataSchemas/$DATA_SCHEMA_KEY?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

In [ ]:
# @title Error: key is not allowed to be updated
CORPUS_ID="" # @param {type: "string"}
DATA_SCHEMA_ID="test" # @param {type: "string"}
DATA_SCHEMA_KEY="verbal" # @param {type: "string"}
DATA_SCHEMA_TYPE="FLOAT" # @param {type: "string"}
DATA_SCHEMA_GRANULARITY="GRANULARITY_PARTITION_LEVEL" # @param {type: "string"}
DATA_SCHEMA_SEARCH_STRATEGY_TYPE="SMART_SEARCH" # @param {type: "string"}

REQUEST="""{
  'key': '%s',
  'schema_details' : {
    'type':'%s',
    'granularity':'%s',
    'search_strategy': {
      'search_strategy_type':'%s'
    }
  }
}"""%(DATA_SCHEMA_KEY, DATA_SCHEMA_TYPE, DATA_SCHEMA_GRANULARITY,DATA_SCHEMA_SEARCH_STRATEGY_TYPE)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/dataSchemas/$DATA_SCHEMA_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

## 删除数据模式

In [ ]:
# @title Please run other sections first which requires a valid Data Schema before deleting the data schema.
CORPUS_ID="" # @param {type: "string"}
DATA_SCHEMA_ID="test" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/dataSchemas/$DATA_SCHEMA_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X DELETE

# 注释管理

In [ ]:
# @title CreateAnnotation (System generates asset id with primitive data type on partition level), the corresponding data schema needs to be created in advance with annotation key.
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
ANNOTATION_KEY="" # @param {type: "string"}
ANNOTATION_STRING_VALUE="STRING" # @param {type: "string"}
ANNOTATION_START_OFFSET_SECONDS=1 # @param {type: "integer"}
ANNOTATION_START_OFFSET_NANO_SECONDS=0 # @param {type: "integer"}
ANNOTATION_END_OFFSET_SECONDS=5 # @param {type: "integer"}
ANNOTATION_END_OFFSET_NANO_SSECONDS=0 # @param {type: "integer"}

REQUEST="""{
  'user_specified_annotation' : {
    'key':'%s',
    'value': {
      'str_value':'%s'
    },
    'partition': {
      'relative_temporal_partition': {
        'start_offset': {
          'seconds': %d,
          'nanos': %d
        },
        'end_offset': {
          'seconds': %d,
          'nanos': %d
        }
      }
    }
  }
}"""%(ANNOTATION_KEY, ANNOTATION_STRING_VALUE, ANNOTATION_START_OFFSET_SECONDS,ANNOTATION_START_OFFSET_NANO_SECONDS,ANNOTATION_END_OFFSET_SECONDS,ANNOTATION_END_OFFSET_NANO_SSECONDS)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/annotations \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" -X POST -d "$REQUEST"

In [ ]:
# @title CreateAnnotation (User specified asset id with customized struct on asset level), the corresponding data schema needs to be created in advance with annotation key.
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
ANNOTATION_ID="" # @param {type: "string"}
ANNOTATION_KEY="" # @param {type: "string"}

# Corresponding create data schema request for the annotation, could be skipped if it already exist.
CREATE_DATA_SCHEMA_REQUEST="""{
  'key': '%s',
  'schema_details' : {
    'type':'CUSTOMIZED_STRUCT',
    'granularity':'GRANULARITY_ASSET_LEVEL',
    'customized_struct_config': {
      'field_schemas': {
         'name': {
            'type':'STRING',
            'granularity':'GRANULARITY_ASSET_LEVEL',
            'search_strategy': {
               'search_strategy_type':'SMART_SEARCH'
            }
         },
         'score': {
            'type':'FLOAT',
            'granularity':'GRANULARITY_ASSET_LEVEL',
            'search_strategy': {
               'search_strategy_type':'EXACT_SEARCH'
            }
         }
      }
    }
  }
}"""%(ANNOTATION_KEY)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/dataSchemas \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" -X POST -d "$CREATE_DATA_SCHEMA_REQUEST"

CREATE_ANNOTATION_REQUEST="""{
  'user_specified_annotation' : {
    'key':'%s',
    'value': {
      'customized_struct_value':{
        'elements' : {
          'name': {
            'str_value':'intro 101'
          },
          'score': {
            'float_value':0.98
          }
        }
      }
    }
  }
}"""%(ANNOTATION_KEY)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/annotations?annotation_id={ANNOTATION_ID} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" -X POST -d "$CREATE_ANNOTATION_REQUEST"

In [ ]:
# @title GetAnnotation
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
ANNOTATION_ID="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/annotations/$ANNOTATION_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

## 列出注释

In [ ]:
# @title List all annotations under one asset.
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/annotations \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List all annotations under one asset with page size specified.
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
PAGE_SIZE=2 # @param {type: "integer"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/annotations?'page_size='{PAGE_SIZE} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List all annotations under one asset with page size and page token specified.
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
PAGE_SIZE=2 # @param {type: "integer"}
PAGE_TOKEN="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/annotations?'page_size='{PAGE_SIZE}'&page_token='{PAGE_TOKEN} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List all annotations under one asset filter by key.
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
ANNOTATION_KEY="player" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/annotations?'filter=key='$ANNOTATION_KEY \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List all annotations under one asset filter by relative temporal partition.
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
ANNOTATION_START_TIME="3s" # @param {type: "string"}
ANNOTATION_END_TIME="5s" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/annotations?'filter=partition.relative_temporal_partition.start_offset>'$ANNOTATION_START_TIME'%20AND%20partition.relative_temporal_partition.end_offset<'$ANNOTATION_END_TIME \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

更新注释

In [ ]:
# @title Use update mask * to do a full replacement
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
ANNOTATION_ID="" # @param {type: "string"}
ANNOTATION_KEY="" # @param {type: "string"}
ANNOTATION_STRING_VALUE="" # @param {type: "string"}
ANNOTATION_START_OFFSET_SECONDS=3 # @param {type: "integer"}
ANNOTATION_START_OFFSET_NANO_SECONDS=0 # @param {type: "integer"}
ANNOTATION_END_OFFSET_SECONDS=7 # @param {type: "integer"}
ANNOTATION_END_OFFSET_NANO_SSECONDS=0 # @param {type: "integer"}
UPDATE_MASK="*"

REQUEST="""{
  'user_specified_annotation' : {
    'key':'%s',
    'value': {
      'str_value':'%s'
    },
    'partition': {
      'relative_temporal_partition': {
        'start_offset': {
          'seconds': %d,
          'nanos': %d
        },
        'end_offset': {
          'seconds': %d,
          'nanos': %d
        }
      }
    }
  }
}"""%(ANNOTATION_KEY, ANNOTATION_STRING_VALUE, ANNOTATION_START_OFFSET_SECONDS,ANNOTATION_START_OFFSET_NANO_SECONDS,ANNOTATION_END_OFFSET_SECONDS,ANNOTATION_END_OFFSET_NANO_SSECONDS)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/annotations/$ANNOTATION_ID?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

In [ ]:
# @title Use update mask to do partial replacement
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
ANNOTATION_ID="" # @param {type: "string"}
UPDATE_MASK="user_specified_annotation.value" # @param {type: "string"}

REQUEST="""{
  'user_specified_annotation' : {
    'key':'random',
    'value': {
      'customized_struct_value':{
        'elements' : {
          'name': {
            'str_value':'math'
          },
          'score': {
            'float_value':0.67
          }
        }
      }
    }
  }
}"""

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/annotations/$ANNOTATION_ID?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

## 删除注释

In [ ]:
# @title Please run other sections first which requires a valid Annotation before deleting the annotation.
CORPUS_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
ANNOTATION_ID="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/assets/$ASSET_ID/annotations/$ANNOTATION_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X DELETE

# 收藏管理

创建集合

In [ ]:
# @title System generates collection Id.
CORPUS_ID="" # @param {type: "string"}
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" -X POST

In [ ]:
# @title User specified collection Id.
CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections?collection_id={COLLECTION_ID} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" -X POST

更新集合

In [ ]:
# @title Use update mask * to do a full replacement.
CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
UPDATE_MASK="*"
REQUEST="""{
    'display_name': 'test-update',
    'description': 'update description'
}"""
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

In [ ]:
# @title Update diaplay_name only.
CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
UPDATE_MASK="display_name"
REQUEST="""{
    'display_name': 'test-update-2'
}"""
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

In [ ]:
# @title Update description only.
CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
UPDATE_MASK="description"
REQUEST="""{
    'description': 'update description 2'
}"""
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

In [ ]:
# @title Expire_time is output only, not allowed to be updated.
CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
UPDATE_MASK="expire_time"
REQUEST="""{
    'expire_time': {'seconds':'1688163855'}
}"""

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID?update_mask={UPDATE_MASK} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X PATCH \
  -d "$REQUEST"

获取集合

In [ ]:
CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

## 列出集合 (ListCollections)

In [ ]:
# @title List all collections under one corpus.
CORPUS_ID="" # @param {type: "string"}

!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List all collections with page size specified.
CORPUS_ID="" # @param {type: "string"}
PAGE_SIZE=2 # @param {type: "integer"}
!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections?'page_size='{PAGE_SIZE} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List collections with page token specified.
CORPUS_ID="" # @param {type: "string"}
PAGE_SIZE=2 # @param {type: "integer"}
PAGE_TOKEN="" # @param {type: "string"}
!curl $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections?'page_size='{PAGE_SIZE}'&page_token='{PAGE_TOKEN} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

添加集合项

In [ ]:
# @title Please run other sections first which requires a valid Asset and Collection before adding a collection item.

CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
REQUEST="""{
    item: {
      'type': 'ASSET',
      'item_resource': 'projects/%d/locations/%s/corpora/%s/assets/%s'
    }
}"""%(PROJECT_NUMBER, LOCATION_ID, CORPUS_ID, ASSET_ID)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID:addCollectionItem \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X POST \
  -d "$REQUEST"

In [ ]:
# @title Item already added to collection, will succeed.
CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
REQUEST="""{
    item: {
      'type': 'ASSET',
      'item_resource': 'projects/%d/locations/%s/corpora/%s/assets/%s'
    }
}"""%(PROJECT_NUMBER, LOCATION_ID, CORPUS_ID, ASSET_ID)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID:addCollectionItem \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X POST \
  -d "$REQUEST"

查看收藏物品

In [ ]:
# @title List all items in a collection.
CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID:viewCollectionItems \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List all items with page size specified.
CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
PAGE_SIZE=2 # @param {type: "integer"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID:viewCollectionItems?'page_size='{PAGE_SIZE} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

In [ ]:
# @title List all items with page token specified.
CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
PAGE_SIZE=2 # @param {type: "integer"}
PAGE_TOKEN="" # @param {type: "string"}

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID:viewCollectionItems?'page_size='{PAGE_SIZE}'&page_token='{PAGE_TOKEN} \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8"

## 删除集合项

In [ ]:
# @title  Remove an item from a collection

CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
REQUEST="""{
    item: {
      'type': 'ASSET',
      'item_resource': 'projects/%d/locations/%s/corpora/%s/assets/%s'
    }
}"""%(PROJECT_NUMBER, LOCATION_ID, CORPUS_ID, ASSET_ID)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID:removeCollectionItem \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X POST \
  -d "$REQUEST"

In [ ]:
# @title Item already removed from a collection, will fail with a FAILED_PRECONDITION error.
CORPUS_ID="" # @param {type: "string"}
COLLECTION_ID="" # @param {type: "string"}
ASSET_ID="" # @param {type: "string"}
REQUEST="""{
    item: {
      'type': 'ASSET',
      'item_resource': 'projects/%d/locations/%s/corpora/%s/assets/%s'
    }
}"""%(PROJECT_NUMBER, LOCATION_ID, CORPUS_ID, ASSET_ID)

!curl \
  $ENDPOINT_WITH_VERSION/projects/$PROJECT_NUMBER/locations/$LOCATION_ID/corpora/$CORPUS_ID/collections/$COLLECTION_ID:removeCollectionItem \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -X POST \
  -d "$REQUEST"